In [1]:
import requests
import os
import time
import zipfile
import shutil
from datetime import datetime
import gc

# Diretório para salvar os arquivos
output_dir = 'dados_anp_ca'
os.makedirs(output_dir, exist_ok=True)

# Base URL
base_url = 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/'

# Anos de 2004 a 2025
anos = range(2004, 2026)  # Até 2025 inclusive

# Semestres: apenas 01 (1º semestre) e 02 (2º semestre)
semestres = ['01', '02']

def download_file(url, filepath, chunk_size=8192, max_retries=5, timeout=60):
    """
    Baixa o arquivo em chunks com retry em caso de falha para economizar memória e persistir no download.
    """
    for attempt in range(max_retries):
        try:
            # Remove arquivo parcial se existir de tentativas anteriores
            if os.path.exists(filepath):
                os.remove(filepath)
            
            print(f"Tentativa {attempt + 1}/{max_retries} para {url}")
            
            response = requests.get(url, stream=True, timeout=timeout)
            response.raise_for_status()  # Levanta exceção para erros HTTP
            
            total_size = int(response.headers.get('content-length', 0))
            downloaded_size = 0
            
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=chunk_size):
                    if chunk:
                        f.write(chunk)
                        downloaded_size += len(chunk)
                        # Opcional: progresso se quiser, mas mantém simples
                        if total_size > 0:
                            progress = (downloaded_size / total_size) * 100
                            print(f"\rProgresso: {progress:.1f}%", end='', flush=True)
            
            # Verifica se o arquivo foi baixado completamente (tamanho > 0)
            if os.path.getsize(filepath) > 0:
                print(f"\nDownload concluído: {os.path.basename(filepath)} (tamanho: {os.path.getsize(filepath)} bytes)")
                return True
            else:
                print(f"\nArquivo vazio após download: {filepath}. Tentando novamente...")
                os.remove(filepath)
                continue
                
        except requests.exceptions.RequestException as e:
            print(f"\nTentativa {attempt + 1} falhou: {e}")
            if os.path.exists(filepath):
                os.remove(filepath)
            if attempt < max_retries - 1:
                wait_time = (2 ** attempt) + 1  # Backoff exponencial: 3s, 5s, 9s, etc.
                print(f"Aguardando {wait_time}s antes da próxima tentativa...")
                time.sleep(wait_time)
            else:
                print(f"Falha após {max_retries} tentativas para {url}")
                return False
        except Exception as e:
            print(f"\nErro inesperado na tentativa {attempt + 1}: {e}")
            if os.path.exists(filepath):
                os.remove(filepath)
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return False
    
    return False

# Loop principal
sucessos = 0
falhas = 0

for ano in anos:
    for semestre in semestres:
        # Para 2025, só tenta até 01 (como mencionado), pula 02 se for o caso
        if ano == 2025 and semestre == '02':
            print(f"Pulando 2025-02 (ainda não disponível).")
            continue
        
        # Construir URL baseado no ano
        if ano < 2022:
            # Arquivos antigos: CSV direto
            url = f"{base_url}ca-{ano}-{semestre}.csv"
            is_zip = False
        elif ano == 2022 and semestre == '01':
            # Especial: ZIP com nome diferente
            url = f"{base_url}precos-semestrais-ca.zip"
            is_zip = True
        else:
            # Recentes: ZIP
            url = f"{base_url}ca-{ano}-{semestre}.zip"
            is_zip = True
        
        filename_csv = f"ca-{ano}-{semestre}.csv"
        filepath_csv = os.path.join(output_dir, filename_csv)
        
        if os.path.exists(filepath_csv) and os.path.getsize(filepath_csv) > 0:
            print(f"Arquivo já existe e é válido: {filepath_csv}. Pulando.")
            sucessos += 1
            continue
        
        temp_path = None
        if is_zip:
            temp_filename = f"temp_ca-{ano}-{semestre}.zip"
            temp_path = os.path.join(output_dir, temp_filename)
            print(f"Baixando ZIP: {url}")
            success = download_file(url, temp_path)
            
            if success:
                try:
                    # Diretório temporário para extração
                    extract_dir = os.path.join(output_dir, f'temp_extract_{ano}_{semestre}')
                    os.makedirs(extract_dir, exist_ok=True)
                    
                    with zipfile.ZipFile(temp_path, 'r') as zip_ref:
                        zip_ref.extractall(extract_dir)
                    
                    # Encontrar o arquivo CSV extraído
                    csv_found = None
                    for root, dirs, files in os.walk(extract_dir):
                        for file in files:
                            if file.endswith('.csv'):
                                csv_found = os.path.join(root, file)
                                break
                        if csv_found:
                            break
                    
                    if csv_found and os.path.getsize(csv_found) > 0:
                        # Copiar e renomear para o padrão
                        shutil.copy2(csv_found, filepath_csv)
                        print(f"Extraído e renomeado com sucesso: {filepath_csv}")
                        success = True
                    else:
                        print(f"Nenhum CSV válido encontrado no ZIP para {ano}-{semestre}")
                        success = False
                    
                    # Limpar temporários
                    shutil.rmtree(extract_dir)
                    os.remove(temp_path)
                    
                except zipfile.BadZipFile:
                    print(f"Arquivo ZIP inválido: {temp_path}")
                    success = False
                    if os.path.exists(temp_path):
                        os.remove(temp_path)
                except Exception as e:
                    print(f"Erro ao extrair ZIP para {ano}-{semestre}: {e}")
                    success = False
                    if os.path.exists(temp_path):
                        os.remove(temp_path)
        else:
            # Download direto para CSV
            print(f"Baixando CSV: {url}")
            success = download_file(url, filepath_csv)
        
        if success:
            sucessos += 1
            # Limpa memória após cada download/extrair
            gc.collect()
        else:
            falhas += 1
            print(f"Falha final em {ano}-{semestre}. Arquivo não foi baixado/extraído.")

print(f"\nDownload e extração concluídos! Sucessos: {sucessos}, Falhas: {falhas}")

Arquivo já existe e é válido: dados_anp_ca\ca-2004-01.csv. Pulando.
Arquivo já existe e é válido: dados_anp_ca\ca-2004-02.csv. Pulando.
Arquivo já existe e é válido: dados_anp_ca\ca-2005-01.csv. Pulando.
Arquivo já existe e é válido: dados_anp_ca\ca-2005-02.csv. Pulando.
Arquivo já existe e é válido: dados_anp_ca\ca-2006-01.csv. Pulando.
Arquivo já existe e é válido: dados_anp_ca\ca-2006-02.csv. Pulando.
Arquivo já existe e é válido: dados_anp_ca\ca-2007-01.csv. Pulando.
Arquivo já existe e é válido: dados_anp_ca\ca-2007-02.csv. Pulando.
Arquivo já existe e é válido: dados_anp_ca\ca-2008-01.csv. Pulando.
Arquivo já existe e é válido: dados_anp_ca\ca-2008-02.csv. Pulando.
Arquivo já existe e é válido: dados_anp_ca\ca-2009-01.csv. Pulando.
Arquivo já existe e é válido: dados_anp_ca\ca-2009-02.csv. Pulando.
Arquivo já existe e é válido: dados_anp_ca\ca-2010-01.csv. Pulando.
Arquivo já existe e é válido: dados_anp_ca\ca-2010-02.csv. Pulando.
Arquivo já existe e é válido: dados_anp_ca\ca-20